# Part 1 Training

## 1. Data Preparation

### 1.1 Data import

In [ ]:
import pandas as pd
import os
#from google.colab import drive
#drive.mount('/content/drive')
#file_path = '/content/drive/MyDrive/fiisual/archive/all_data.csv'

current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
file_path = os.path.join(project_root, 'data', 'archive', 'all_data.csv')


df = pd.read_csv(file_path, encoding='latin-1', names=['sentiment', 'text'])

# 檢查資料
print(f"原始資料筆數: {len(df)}")
print(df['sentiment'].value_counts())

✅ 成功找到檔案！
原始資料筆數: 4846
sentiment
neutral     2879
positive    1363
negative     604
Name: count, dtype: int64


### 1.2 transformer and cut

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict


# Label Mapping: 把文字轉成數字 (FinBERT 通常順序是: positive, negative, neutral，或是自定義)
# 這裡我們自定義 mapping，確保自己知道哪個數字代表什麼
label_map = {'positive': 2, 'neutral': 1, 'negative': 0}
df['label'] = df['sentiment'].map(label_map)

# 切割資料集: 80% 訓練, 10% 驗證, 10% 測試
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

# 轉成 Hugging Face 的 Dataset 格式
dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(val_df),
    'test': Dataset.from_pandas(test_df)
})

### 1.3 Tokenization

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

model_name = "ProsusAI/finbert"  # 指定使用金融專用的 FinBERT
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

## 2. Modeling

### 2.1 setting Training Arguments

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# 載入預訓練模型 (num_labels=3 代表正/中/負)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 解決 FinBERT label 對應問題 (確保 output 對得上我們的 0,1,2)
model.config.id2label = {0: 'negative', 1: 'neutral', 2: 'positive'}
model.config.label2id = {'negative': 0, 'neutral': 1, 'positive': 2}

# 定義評估指標 (Accuracy & F1)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {'accuracy': acc, 'f1': f1}

# 設定訓練參數
training_args = TrainingArguments(
    output_dir='./results',          # 輸出路徑
    num_train_epochs=3,              # 訓練幾輪 (通常 3-5 輪就夠了)
    per_device_train_batch_size=16,  # 批次大小 (如果顯卡記憶體不夠可調小)
    per_device_eval_batch_size=32,
    eval_strategy="epoch",     # 每個 epoch 結束驗證一次
    save_strategy="epoch",           # 每個 epoch 存一次檔
    learning_rate=2e-5,              # 微調通常用很小的學習率
    weight_decay=0.01,
    load_best_model_at_end=True,     # 訓練完載入表現最好的模型
    metric_for_best_model="accuracy",
    report_to=[]                     # 避免wandb的初始化問題
)


### 2.2 Start Training

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)

print("開始訓練 FinBERT...")
trainer.train()

## 3 Result

In [ ]:
# 在測試集上跑分
test_results = trainer.evaluate(tokenized_datasets['test'])
print(f"測試集結果: {test_results}")

# 儲存模型 (這就是你的資產！之後爬蟲下來的資料都靠它)
model.save_pretrained("./my_finbert_model")
tokenizer.save_pretrained("./my_finbert_model")

print("模型已儲存至 ./my_finbert_model")

## 4. export the model

In [ ]:
# import shutil
# source_dir = './my_finbert_model'
# models_dir = '/content/drive/MyDrive/fiisual/my_finbert_model_backup'
# shutil.copytree(source_dir, destination_dir)

from pathlib import Path

# 1. 設定路徑 (根據上一則對話的邏輯)
# 假設你在 notebooks 資料夾內
project_root = Path.cwd().parent 
models_dir = project_root / 'models' / 'finbert_sentiment'

# 2. 直接存到專案結構的 models 資料夾
print(f"💾 正在儲存模型至: {models_dir}")
model.save_pretrained(models_dir)
tokenizer.save_pretrained(models_dir)

print("✅ 模型儲存完成！")